In [42]:
import numpy as np
import pickle

In [2]:
import pandas as pd
df = pd.read_csv('bike-no.csv')
df.head()

,instant,date,season,yr,mnth,weekday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,1/1/2011,1,0,1,6,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,1/2/2011,1,0,1,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,1/3/2011,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,1/4/2011,1,0,1,2,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,1/5/2011,1,0,1,3,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [3]:
df['date'] = pd.to_datetime(df['date'])

In [4]:
df.shape[0]

731

In [5]:
train_data = df.iloc[:650]
test_data = df.iloc[650:]

In [7]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters

In [18]:
forecasting_parameters = ForecastingParameters(time_column_name='date', 
                                               forecast_horizon=50,
                                               
                                               freq='D',
                                               target_lags='auto',
                                               target_rolling_window_size=10)

In [19]:
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
import logging

In [20]:
automl_config = AutoMLConfig(task='forecasting',
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=15,
                             enable_early_stopping=True,
                             training_data=train_data,
                             label_column_name='registered',
                             n_cross_validations=5,
                             enable_ensembling=False,
                             verbosity=logging.INFO,
                             forecasting_parameters = forecasting_parameters)

In [21]:
ws = Workspace.from_config()
experiment = Experiment(ws, "Tutorial-automl-forecasting")
local_run = experiment.submit(automl_config, show_output=True)
best_run, fitted_model = local_run.get_output()

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
Tutorial-automl-forecasting,AutoML_ad25368e-6d9b-421c-9d9f-268a20e9cbd9,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Heuristic parameters: Target_Lag = '[1]'.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Curr

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [22]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Tutorial-automl-forecasting,AutoML_ad25368e-6d9b-421c-9d9f-268a20e9cbd9_5,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [23]:
fitted_model

ForecastingPipelineWrapper(pipeline=Pipeline(memory=None,
                                             steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(country_or_region=None, drop_column_names=[], featurization_config=FeaturizationConfig(
    blocked_transformers=None,
    column_purposes=None,
    transformer_params=None,
    dataset_language=None,
    drop_columns=None,
    prediction_transform_type=None
), force_time_...
                                   1.1221852630251072e-11,
                                   1.1221852630251072e-11,
                                   1.1144928926689601e-11,
                                   1.129532370851668e-11,
                                   1.1221852630251072e-11,
                                   1.1159763125890436e-11,
                                   1.1796815937189213e-11,
                                   1.143795810804476e-11, 1.143795810804476e-11,
                   

In [40]:
test_labels = test_data['registered'].to_numpy()
test_labels

array([6222, 4857, 4559, 5115, 6612, 6482, 6501, 4671, 5284, 4692, 6228,
       6625, 6898, 6484, 6262, 5209, 3461,   20, 1009, 5147, 5520, 5229,
       4109, 3906, 4881, 5220, 4709, 4975, 5283, 4446, 4562, 5172, 3767,
       5122, 5125, 5214, 4316, 3747, 5050, 5100, 4531, 1470, 2307, 1745,
       2115, 4750, 3836, 5062, 5080, 5306, 4240, 3757, 5679, 6055, 5398,
       5035, 4659, 4429, 2787, 4841, 5219, 5009, 5107, 5182, 4280, 3248,
       4373, 5124, 4934, 3814, 3402, 1544, 1379,  746,  573,  432, 1867,
       2451, 1182, 1432, 2290])

In [41]:
label_query = test_labels.copy().astype(np.float)
label_query.fill(np.nan)
label_query

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan])

In [39]:
fitted_model.quantiles = [0.05,0.5, 0.9]
fitted_model.forecast_quantiles(
    test_data,label_query)

Cannot reset time index. Use reindex to change its values.
Cannot reset time index. Use reindex to change its values.


,date,0.05,0.5,0.9
0,2012-10-12,6222.00,6222.00,6222.00
1,2012-10-13,4857.00,4857.00,4857.00
2,2012-10-14,4559.00,4559.00,4559.00
3,2012-10-15,5115.00,5115.00,5115.00
4,2012-10-16,6612.00,6612.00,6612.00
...,...,...,...,...
76,2012-12-27,1867.00,1867.00,1867.00
77,2012-12-28,2451.00,2451.00,2451.00
78,2012-12-29,1182.00,1182.00,1182.00
79,2012-12-30,1432.00,1432.00,1432.00


In [43]:
fit_model_2 = pickle.load(open('model.pkl','rb'))

unclosed file <_io.BufferedReader name='model.pkl'>


In [45]:
fit_model_2.quantiles = [0.05,0.5, 0.9]
fit_model_2.forecast_quantiles(
    test_data)

,date,0.05,0.5,0.9
0,2012-10-12,6222.00,6222.00,6222.00
1,2012-10-13,4857.01,4857.01,4857.01
2,2012-10-14,4559.01,4559.01,4559.01
3,2012-10-15,5115.00,5115.01,5115.01
4,2012-10-16,6612.00,6612.00,6612.00
...,...,...,...,...
76,2012-12-27,1867.01,1867.01,1867.02
77,2012-12-28,2451.01,2451.01,2451.01
78,2012-12-29,1182.01,1182.01,1182.02
79,2012-12-30,1432.01,1432.01,1432.02
